In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [2]:
#time module to not get blocked
import time

#function to get all links to relevant jobs
def get_job_links(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    #get last page number
    max_page_text = soup.select(".page-link")[-2].get_text()
    max_page_text = max_page_text.strip()
    max_page = int(max_page_text)
    
    links = []
    
    for i in range(1, max_page + 1):
        r = requests.get(f"{url}&page={i}")
        soup = BeautifulSoup(r.text, "html.parser")
        page_links = ["https://djinni.co" + a["href"] for a in soup.select(".job-item__title-link")]
        links.extend(page_links)
        time.sleep(1)
        
    return links

In [5]:
url = "https://djinni.co/jobs/?primary_keyword=Data%20Analyst&exp_level=no_exp&exp_level=1y"

In [7]:
links = get_job_links(url)

In [8]:
def job_description(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    #find if job is remote, hybrid or office-based
    try:
        arrangement_type = soup.find(class_="d-block font-weight-600", string=re.compile(r"(Office|Remote)", re.IGNORECASE))
        arrangement_type = arrangement_type.get_text()
    except AttributeError:
        arrangement_type = None
    #find if job is fulltime or part-time
    employment_type = soup.find(string=re.compile(r"Employment:"))
    if employment_type is not None:
        employment_type = employment_type.split("Employment: ")[1]
    
    #job domain
    domain = soup.find(string=re.compile(r"Domain:"))
    if domain is not None:
        domain = domain.split("Domain: ")[1]
    
    #English requirements
    language = soup.find(class_="fw-semibold", string=re.compile(r"English"))
    if language is not None:
        language = language.find_next("td").get_text()
    
    #skills requirements
    requirements = soup.select(".row.gx-2.text-secondary.fs-5")
    if requirements:
        requirements = requirements[0].find("div", class_="col").get_text()
    else:
        requirements = "Not Listed"
    
    return [url, arrangement_type, employment_type, domain, language, requirements]

In [14]:
job_descriptions = [] 
for link in links:
    a = job_description(link)
    job_descriptions.append(a)
    time.sleep(3)


['https://djinni.co/jobs/734337-web-analyst-trainee/', 'Full Remote', 'Fulltime', 'Gambling', 'B2 - Upper Intermediate', 'Not Listed']
['https://djinni.co/jobs/736618-acquisition-data-analyst/', 'Office or Remote', 'Fulltime', 'E-commerce / Marketplace', 'B2 - Upper Intermediate', 'SQL, Excel, Data Analysis, Data Visualisation, Data Analyst, Analysis and research, Microsoft Office']
['https://djinni.co/jobs/740506-anti-fraud-analyst/', 'Full Remote', 'Fulltime', 'Gambling', 'A2 - Elementary', 'Not Listed']
['https://djinni.co/jobs/751946-data-analyst/', 'Office or Remote', 'Fulltime', 'Fintech', 'A2 - Elementary', 'Not Listed']
['https://djinni.co/jobs/752898-big-data-analyst/', 'Full Remote', 'Fulltime', 'Telecom / Communications', None, 'SQL, Jira, Confluence']
['https://djinni.co/jobs/756520-data-analyst/', 'Full Remote', 'Fulltime', 'Healthcare / MedTech', 'B1 - Intermediate', 'Not Listed']
['https://djinni.co/jobs/760743-anti-fraud-analyst/', 'Full Remote', 'Fulltime', 'Gambling',

In [36]:
import pandas as pd
df = pd.DataFrame(job_descriptions, columns=["url", "arrangement_type", "employment_type", "domain", "English level", "requirements"])
df.to_csv("jobs.csv", index=False)

,url,arrangement_type,employment_type,domain,English level,requirements
0,https://djinni.co/jobs/734337-web-analyst-trai...,Full Remote,Fulltime,Gambling,B2 - Upper Intermediate,Not Listed
1,https://djinni.co/jobs/736618-acquisition-data...,Office or Remote,Fulltime,E-commerce / Marketplace,B2 - Upper Intermediate,"SQL, Excel, Data Analysis, Data Visualisation,..."
2,https://djinni.co/jobs/740506-anti-fraud-analyst/,Full Remote,Fulltime,Gambling,A2 - Elementary,Not Listed
3,https://djinni.co/jobs/751946-data-analyst/,Office or Remote,Fulltime,Fintech,A2 - Elementary,Not Listed
4,https://djinni.co/jobs/752898-big-data-analyst/,Full Remote,Fulltime,Telecom / Communications,None,"SQL, Jira, Confluence"
5,https://djinni.co/jobs/756520-data-analyst/,Full Remote,Fulltime,Healthcare / MedTech,B1 - Intermediate,Not Listed
6,https://djinni.co/jobs/760743-anti-fraud-analyst/,Full Remote,Fulltime,Gambling,None,Not Listed
7,https://djinni.co/jobs/762153-junior-business-...,Full Remote,Fulltime,Other,B1 - Intermediate,"business analysis, Business Intelligence, Busi..."
8,https://djinni.co/jobs/762155-business-intelli...,Full Remote,Fulltime,Other,B1 - Intermediate,"Business Intelligence, Busines Analysis, busin..."
9,https://djinni.co/jobs/762452-retention-and-pr...,Hybrid Remote,Fulltime,Advertising / Marketing,B2 - Upper Intermediate,Not Listed
